In [1]:
%matplotlib tk

import argparse
import gym
import datetime
import os
import random
import tempfile
import numpy as np
import pickle

import ray
from ray import tune
from ray.tune.logger import Logger, UnifiedLogger, pretty_print
from ray.rllib.env.multi_agent_env import make_multi_agent
from ray.rllib.examples.models.shared_weights_model import TF2SharedWeightsModel
from ray.rllib.models import ModelCatalog
from ray.rllib.utils.framework import try_import_tf
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.rllib.agents.ppo import ppo, PPOTrainer, PPOTFPolicy
from ray.rllib.agents.a3c.a3c_tf_policy import A3CTFPolicy
from ray.rllib.agents.a3c import a3c
from ray.rllib.models import ModelCatalog
from ray.rllib.policy.policy import PolicySpec
from environment_rllib_3d import MyEnv
from settings.initial_settings import *
from settings.reset_conditions import reset_conditions
#from modules.models import MyConv2DModel_v0B_Small_CBAM_1DConv_Share
#from modules.models import MyRNNUAVClass
#from modules.models import DenseNetModelLarge
from tensorflow.keras.utils import plot_model
from modules.savers import save_conditions
from utility.result_env import render_env
from utility.terminate_uavsimproc import teminate_proc
from utility.latest_learned_file_path import latest_learned_file_path
from utility.save_logs import save_logs

import matplotlib.pyplot as plt
import matplotlib
import tensorflow as tf
import cv2
import ctypes
import warnings

#UCAV.exeが起動している場合、プロセスキルする。
teminate_proc.UAVsimprockill(proc_name="UCAV.exe")

warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 
warnings.filterwarnings('ignore', category=matplotlib.MatplotlibDeprecationWarning)
np.set_printoptions(precision=3, suppress=True)
PROJECT = "UCAV"
TRIAL_ID = 2
TRIAL = 'test_' + str(TRIAL_ID)
EVAL_FREQ = 25
CONTINUAL = False
NUM_EVAL = 1
def custom_log_creator(custom_path, custom_str):
    timestr = datetime.datetime.today().strftime("%Y-%m-%d_%H-%M-%S")
    logdir_prefix = "{}_{}".format(custom_str, timestr)

    def logger_creator(config):
        if not os.path.exists(custom_path):
            os.makedirs(custom_path)
        logdir = tempfile.mkdtemp(prefix=logdir_prefix, dir=custom_path)
        return UnifiedLogger(config, logdir, loggers=None)

    return logger_creator

ray.shutdown()
ray.init(ignore_reinit_error=True, log_to_driver=False)

#ModelCatalog.register_custom_model('my_model', MyRNNUAVClass)

# config = {"env": MyEnv,
#           "num_workers": NUM_WORKERS,
#           "num_gpus": NUM_GPUS,
#           "num_cpus_per_worker": NUM_CPUS_PER_WORKER,
#           "num_sgd_iter": NUM_SGD_ITER,
#           "lr": LEARNING_RATE,
#           "gamma": GAMMA,  # default=0.99
#           "model": {"custom_model": "my_model"}
#           # "framework": framework
#           }  # use tensorflow 2
eval_env = MyEnv({})
policies = {
    #"blue_1": PolicySpec(config={"gamma": 0.99}),
    #"blue_2": PolicySpec(config={"gamma": 0.95}),
    "blue_0": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space, {}),
    "blue_1": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space, {}),
    #"blue_0": (A3CTFPolicy, eval_env.observation_space, eval_env.action_space, {}),
    #"blue_1": (A3CTFPolicy, eval_env.observation_space, eval_env.action_space, {}),
}
policy_ids = list(policies.keys())

def policy_mapping_fn(agent_id, episode, **kwargs):
    #print(agent_id,episode)
    #pol_id = policy_ids[agent_id]

    pol_id = agent_id
    return pol_id

# Instanciate the evaluation env
config = ppo.DEFAULT_CONFIG.copy()
config = {"env": MyEnv,"num_gpus": 0,"num_workers": 0, "num_cpus_per_worker": 0,"num_gpus_per_worker": 0,
          "train_batch_size": 60,"batch_mode": "complete_episodes",
          "gamma":0.99, "lr": 1e-4,"shuffle_sequences": True,
          #"clip_actions":True,"normalize_actions":True,
          #"observation_space":eval_env.observation_space,"action_space":eval_env.action_space,
          #"explore":True,
          "sgd_minibatch_size": 16, "num_sgd_iter":4,
          #"exploration_config": {"type": "StochasticSampling","random_timesteps":1200*5*0}, #PPO デフォルト "random_timesteps":0
          "model":{"fcnet_activation": "relu","fcnet_hiddens": [256, 256, 256],"post_fcnet_activation": "linear",
                   "vf_share_layers": True,},#"linear","relu","tanh" "use_lstm":True,"lstm_cell_size":256,"max_seq_len":128
          #"model":{"vf_share_layers": True,"use_lstm": True},
          #"model": {"custom_model": "my_model"},
          "multiagent": {"policies": policies,  "policy_mapping_fn": policy_mapping_fn}
         }
#res_name = "sgd"+str(config["sgd_minibatch_size"])+"sgd_num"+str(config["num_sgd_iter"])+"lr"+str(config["lr"])+"gamma"+str(config["gamma"])
res_name = "test"
conditions_dir = os.path.join('./' + PROJECT + '/conditions/')

if not os.path.exists(conditions_dir):
    os.makedirs(conditions_dir)
save_conditions(conditions_dir)

# PPOTrainer()は、try_import_tfを使うと、なぜかTensorflowのeager modeのエラーになる。

trainer = ppo.PPOTrainer(config=config,
                         logger_creator=custom_log_creator(
                             os.path.expanduser("./" + PROJECT + "/logs"), TRIAL))

if CONTINUAL:
    # Continual learning: Need to specify the checkpoint
    # model_path = PROJECT + '/checkpoints/' + TRIAL + '/checkpoint_000197/checkpoint-197'
    model_path = latest_learned_file_path('./UCAV/checkpoints/test_2/*')
    trainer.restore(checkpoint_path=model_path)

models_dir = os.path.join('./' + PROJECT + '/models/')
if not os.path.exists(models_dir):
    os.makedirs(models_dir)
for j in range(eval_env.blue_num):
    text_name = models_dir + TRIAL + "blue_"+str(j) +'.txt'
    with open(text_name, "w") as fp:
        trainer.get_policy("blue_"+str(j)).model.base_model.summary(print_fn=lambda x: fp.write(x + "\r\n"))
    png_name = models_dir + TRIAL + '.png'
    plot_model(trainer.get_policy("blue_"+str(j)).model.base_model, to_file=png_name, show_shapes=True)



# Define checkpoint dir
check_point_dir = os.path.join('./' + PROJECT + '/checkpoints/', TRIAL)
if not os.path.exists(check_point_dir):
    os.makedirs(check_point_dir)

C:\Users\Takumi\anaconda3\envs\AI2\lib\site-packages\quaternion\numba_wrapper.py:23: UserWarning: 

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Could not import from numba, which means that some
parts of this code may run MUCH more slowly.  You
may wish to install numba.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

  warnings.warn(warning_text)
2022-01-17 01:25:48,735	INFO trainer.py:723 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also want to then set `eager_tracing=True` in order to reach similar execution speed as with static-graph mode.
2022-01-17 01:25:48,736	WARNING ppo.py:151 -- `train_batch_size` (60) cannot be achieved with your other settings (num_workers=0 num_envs_per_worker=1 rollout_fragment_length=200)! Auto-adjusting `rollout_fragment_length` to 60.
2022-01-17 01:25:48,736	INFO ppo.py:167 -- In multi-agent mode, policies will be optimized sequentially by

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')
('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [ ]:
record_mode = 0
results_dir = os.path.join('./' + PROJECT + '/results/')

if not os.path.exists(results_dir):
    os.makedirs(results_dir)
results_file = results_dir + TRIAL + '.pkl'
for steps in range(10001):
    # Training
    print(f'\n----------------- Training at steps:{steps} start! -----------------')
    eval_env.eval = False
    eval_env.reset()
    results = trainer.train()
    save_logs(res_name,results,steps,CONTINUAL)
    print(pretty_print(results))
    #check_point = trainer.save(checkpoint_dir=check_point_dir)
    # Evaluation
    if steps % EVAL_FREQ == 0:
        print(f'\n-------------- Evaluation at steps:{steps} starting ! --------------')
        #print(pretty_print(results))
        check_point = trainer.save(checkpoint_dir=check_point_dir)
        for i in range(NUM_EVAL):
            # print(f'\nEvaluation {i}:')
            eval_env.eval = True
            obs = eval_env.reset()
            done = False
            
            step_num = 0
            fig = plt.figure(1,figsize=(8.0, 6.0))
            ESC = 0x1B          # ESCキーの仮想キーコード
            trajectory_length = 100
            env_blue_pos = [0]
            env_red_pos = [0]
            env_mrm_pos = [0]
            if record_mode == 0:
                file_name = "test_num" + str(steps) +str(i)
                video = cv2.VideoWriter(file_name+'.mp4',0x00000020,20.0,(800,600))

            while True:
                action_dict = {}
                #for j in range(eval_env.blue_num):
                    #if not eval_env.blue[j].hitpoint == 0:
                    #action_dict['blue_' + str(j)] = trainer.compute_action(obs['blue_' + str(j)])
                    #action_dict['blue_' + str(j)] = trainer.compute_single_action(obs['blue_' + str(j)],policy_id='blue_' + str(j),
                    #                                                   clip_actions=True,explore=False)
                    #action_dict['blue_' + str(j)] = trainer.compute_single_action(obs['blue_' + str(j)],policy_id='blue_' + str(j))
                action_dict0 = trainer.compute_single_action(obs['blue_0'],policy_id='blue_0',explore=False)
                action_dict1 = trainer.compute_single_action(obs['blue_1'],policy_id='blue_1',explore=False)
                obs, rewards, dones, infos = eval_env.step({'blue_0': action_dict0, 'blue_1': action_dict1})
                env_blue_pos_temp, env_red_pos_temp, env_mrm_pos_temp= render_env.copy_from_env(eval_env)
                env_blue_pos.append(env_blue_pos_temp)
                env_red_pos.append(env_red_pos_temp)
                env_mrm_pos.append(env_mrm_pos_temp)
                if step_num == 0:
                    del env_blue_pos[0]
                    del env_red_pos[0]
                    del env_mrm_pos[0]

                hist_blue_pos = np.vstack(env_blue_pos)
                hist_red_pos = np.vstack(env_red_pos)
                hist_mrm_pos = np.vstack(env_mrm_pos)
                plt.clf()
                render_env.rend_3d(eval_env,hist_blue_pos,"b",1)
                render_env.rend_3d(eval_env,hist_red_pos,"r",1)
                render_env.rend_3d(eval_env,hist_mrm_pos,"k",1)
                plt.subplots_adjust(left=-0.1,right=1.1,bottom=-0.1,top=1.1)
                fig.canvas.draw()
                plt.pause(.01)
                if record_mode == 0:
                    img = np.array(fig.canvas.renderer.buffer_rgba())
                    img = cv2.cvtColor(img, cv2.COLOR_RGBA2BGR)
                    video.write(img.astype('uint8'))

                
                step_num = step_num + 1
                
                done = dones["__all__"]
                #print(f'rewards:{rewards}')
                #if record_mode == 0:
                #    img = eval_env.render_movie(file_name,step_num)
                #    video.write(img.astype('unit8'))
                #elif record_mode == 1:
                #    eval_env.render()
                #elif record_mode == 2:
                #    eval_env.render()
                    
                #env_blue_pos_temp, env_red_pos_temp, env_mrm_pos_temp = render_env.copy_from_env(eval_env)
                
                #env_blue_pos.append(env_blue_pos_temp)
                #env_red_pos.append(env_red_pos_temp)
                #env_mrm_pos.append(env_mrm_pos_temp)
                #step_num = step_num + 1
                # エピソードの終了処理
                if dones['__all__']:
                    # print(f'all done at {env.steps}')
                    break
                
            #del env_blue_pos[0]
            #del env_red_pos[0]
            #del env_mrm_pos[0]
            
            #hist_blue_pos = np.vstack(env_blue_pos)
            #hist_red_pos = np.vstack(env_red_pos)
            #hist_mrm_pos = np.vstack(env_mrm_pos)
            
            #f = open(results_file,'wb')
            #pickle.dump(emv_blue_pos,f)
            #pickle.dump(emv_red_pos,f)
            #pickle.dump(emv_mrm_pos,f)
            #f.close()
            
            if record_mode == 0:
                video.release()

ray.shutdown()


----------------- Training at steps:0 start! -----------------
-------------------------- Scene: 0 --------------------------
-------------------------- Scene: 1 --------------------------
53 blue_0 DOWN
DOWN LOSE
blue_0 True False 53 -10.14 -7.942500000000001
blue_1 True False 53 -0.135 4.544326168176532
-------------------------- Scene: 0 --------------------------
19 blue_0 DOWN
DOWN LOSE
blue_0 True False 19 -10.145 -12.754999999999999
blue_1 True False 19 -0.12000000000000001 -2.280000000000001
-------------------------- Scene: 1 --------------------------


2022-01-17 01:26:17,038	WARNING deprecation.py:46 -- DeprecationWarning: `slice` has been deprecated. Use `SampleBatch[start:stop]` instead. This will raise an error in the future!


agent_timesteps_total: 144
custom_metrics: {}
date: 2022-01-17_01-26-17
done: false
episode_len_mean: 36.0
episode_media: {}
episode_reward_max: -3.3981738318234695
episode_reward_mean: -9.216586915911735
episode_reward_min: -15.035000000000002
episodes_this_iter: 2
episodes_total: 2
experiment_id: 9375dc5e1ee74bf6be4b3c087d4b8b30
hostname: DESKTOP
info:
  learner:
    blue_0:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 9.999999747378752e-05
        entropy: 7.1533355712890625
        entropy_coeff: 0.0
        kl: 0.0012635646853595972
        model: {}
        policy_loss: -0.15091539919376373
        total_loss: 59.8476676940918
        vf_explained_var: -0.0014420673251152039
        vf_loss: 59.998329162597656
    blue_1:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 9.999999747378752e-05
        entropy: 7.153587341308594
        entropy_coeff: 0.0
        kl: 0.0

21 blue_1 DOWN
DOWN LOSE
blue_0 True False 21 0.0050000000000000044 0.1050000000000001
blue_1 True False 21 -10.02 -10.42
-------------------------- Scene: 0 --------------------------
66 blue_1 DOWN
DOWN LOSE
blue_0 True False 66 -0.045 -5.669999999999998
blue_1 True False 66 -10.02 -14.020000000000003
-------------------------- Scene: 0 --------------------------
agent_timesteps_total: 758
custom_metrics: {}
date: 2022-01-17_01-27-33
done: false
episode_len_mean: 63.166666666666664
episode_media: {}
episode_reward_max: -3.3981738318234695
episode_reward_mean: -11.206362305303912
episode_reward_min: -19.68999999999999
episodes_this_iter: 2
episodes_total: 6
experiment_id: 9375dc5e1ee74bf6be4b3c087d4b8b30
hostname: DESKTOP
info:
  learner:
    blue_0:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 0.15000000596046448
        cur_lr: 9.999999747378752e-05
        entropy: 7.0023298263549805
        entropy_coeff: 0.0
        kl: 0.049049388617277145
        model: {

23 blue_0 DOWN
DOWN LOSE
blue_0 True False 23 -10.145 -13.335
blue_1 True False 23 -0.12000000000000001 -2.7600000000000016
-------------------------- Scene: 1 --------------------------
72 blue_0 DOWN
DOWN LOSE
blue_0 True False 72 -10.14 -15.342500000000001
blue_1 True False 72 -0.135 -3.8918127736157544
-------------------------- Scene: 1 --------------------------
agent_timesteps_total: 1650
custom_metrics: {}
date: 2022-01-17_01-28-47
done: false
episode_len_mean: 82.5
episode_media: {}
episode_reward_max: -3.3981738318234695
episode_reward_mean: -19.058248660543928
episode_reward_min: -69.89000000000007
episodes_this_iter: 2
episodes_total: 10
experiment_id: 9375dc5e1ee74bf6be4b3c087d4b8b30
hostname: DESKTOP
info:
  learner:
    blue_0:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 0.3375000059604645
        cur_lr: 9.999999747378752e-05
        entropy: 6.958006858825684
        entropy_coeff: 0.0
        kl: 0.0639524906873703
        model: {}
        pol

...

304 blue_1 DOWN
DOWN LOSE
blue_0 True False 304 -0.14500000000000002 -44.08000000000018
blue_1 True False 304 -10.12 -46.47999999999999
-------------------------- Scene: 0 --------------------------
agent_timesteps_total: 3954
custom_metrics: {}
date: 2022-01-17_01-32-31
done: false
episode_len_mean: 85.95652173913044
episode_media: {}
episode_reward_max: -3.3981738318234695
episode_reward_mean: -19.857792451142846
episode_reward_min: -90.56000000000012
episodes_this_iter: 1
episodes_total: 23
experiment_id: 9375dc5e1ee74bf6be4b3c087d4b8b30
hostname: DESKTOP
info:
  learner:
    blue_0:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 1.708593726158142
        cur_lr: 9.999999747378752e-05
        entropy: 6.731382369995117
        entropy_coeff: 0.0
        kl: 0.02643590234220028
        model: {}
        policy_loss: -0.0032820608466863632
        total_loss: 22.73760986328125
        vf_explained_var: -0.22322705388069153
        vf_loss: 22.695722579956055
    b

72 blue_0 DOWN
DOWN LOSE
blue_0 True False 72 -9.995 -8.582500000000001
blue_1 True False 72 -0.02 4.215342371983956
-------------------------- Scene: 0 --------------------------
agent_timesteps_total: 4606
custom_metrics: {}
date: 2022-01-17_01-33-34
done: false
episode_len_mean: 85.29629629629629
episode_media: {}
episode_reward_max: -3.3981738318234695
episode_reward_mean: -18.694680889048204
episode_reward_min: -90.56000000000012
episodes_this_iter: 1
episodes_total: 27
experiment_id: 9375dc5e1ee74bf6be4b3c087d4b8b30
hostname: DESKTOP
info:
  learner:
    blue_0:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 1.9221680164337158
        cur_lr: 9.999999747378752e-05
        entropy: 6.839794158935547
        entropy_coeff: 0.0
        kl: 0.008466939441859722
        model: {}
        policy_loss: -0.23025214672088623
        total_loss: 9.5807466506958
        vf_explained_var: -0.01031460240483284
        vf_loss: 9.794723510742188
    blue_1:
      custom_me

19 blue_1 DOWN
DOWN LOSE
blue_0 True False 19 -0.14500000000000002 -2.7550000000000003
blue_1 True False 19 -10.12 -12.280000000000001
-------------------------- Scene: 1 --------------------------
136 blue_1 DOWN
DOWN LOSE
blue_0 True False 136 -0.14 -9.165000000000004
blue_1 True False 136 -10.135 -17.85574698718652
-------------------------- Scene: 0 --------------------------
agent_timesteps_total: 5714
custom_metrics: {}
date: 2022-01-17_01-34-50
done: false
episode_len_mean: 89.28125
episode_media: {}
episode_reward_max: -3.3981738318234695
episode_reward_mean: -18.565535343484
episode_reward_min: -90.56000000000012
episodes_this_iter: 2
episodes_total: 32
experiment_id: 9375dc5e1ee74bf6be4b3c087d4b8b30
hostname: DESKTOP
info:
  learner:
    blue_0:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 1.9221680164337158
        cur_lr: 9.999999747378752e-05
        entropy: 6.767811298370361
        entropy_coeff: 0.0
        kl: 0.010135778225958347
        model:

86 blue_1 DOWN
DOWN LOSE
blue_0 True False 86 -0.14500000000000002 -12.469999999999976
blue_1 True False 86 -10.12 -20.319999999999993
-------------------------- Scene: 0 --------------------------
agent_timesteps_total: 6376
custom_metrics: {}
date: 2022-01-17_01-35-45
done: false
episode_len_mean: 88.55555555555556
episode_media: {}
episode_reward_max: -3.3981738318234695
episode_reward_mean: -19.438067251523382
episode_reward_min: -90.56000000000012
episodes_this_iter: 1
episodes_total: 36
experiment_id: 9375dc5e1ee74bf6be4b3c087d4b8b30
hostname: DESKTOP
info:
  learner:
    blue_0:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 2.883251905441284
        cur_lr: 9.999999747378752e-05
        entropy: 6.699087619781494
        entropy_coeff: 0.0
        kl: 0.00983239896595478
        model: {}
        policy_loss: 0.08090390264987946
        total_loss: 4.849700927734375
        vf_explained_var: 0.11812543869018555
        vf_loss: 4.740447998046875
    blue_1:

105 blue_1 DOWN
DOWN LOSE
blue_0 True False 105 -0.045 -5.9249999999999945
blue_1 True False 105 -10.02 -13.3
-------------------------- Scene: 0 --------------------------
agent_timesteps_total: 7088
custom_metrics: {}
date: 2022-01-17_01-37-03
done: false
episode_len_mean: 86.4390243902439
episode_media: {}
episode_reward_max: -3.3981738318234695
episode_reward_mean: -20.03196148914248
episode_reward_min: -90.56000000000012
episodes_this_iter: 1
episodes_total: 41
experiment_id: 9375dc5e1ee74bf6be4b3c087d4b8b30
hostname: DESKTOP
info:
  learner:
    blue_0:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 2.883251905441284
        cur_lr: 9.999999747378752e-05
        entropy: 6.756227970123291
        entropy_coeff: 0.0
        kl: 0.018126055598258972
        model: {}
        policy_loss: 0.09251461178064346
        total_loss: 42.83626174926758
        vf_explained_var: -0.12962563335895538
        vf_loss: 42.69148635864258
    blue_1:
      custom_metrics: {}